# Worksheet 08

Name:  
UID: 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.571425061036498, 4.084481410417218, 5.71187504883994, 4.870290673001325, 5.206220910766344, 5.899532887812484, 4.427300652829605, 6.732987072061121, 3.834657887270695, 5.571719292131711]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
mean = 8
variance = 1
c2 = np.random.normal(mean, np.sqrt(variance), 10).tolist()
print(c2)


[7.70641928914293, 7.2879378185693104, 8.070172811242438, 6.61864237271875, 8.225452715904622, 6.521694737370276, 8.299789128109644, 5.733746343740268, 5.674618503816273, 7.864772126510419]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
import random

data = []

# Simulate 10 coin tosses
for _ in range(10):
    coin_output = random.choice(['H', 'T'])  # 'H' for heads, 'T' for tails
    if coin_output == 'H':
        # If heads, pick the last data point from c1 and remove it
        if c1:
            p1 = c1.pop()
            data.append(p1)
    else:
        # If tails, pick the last data point from c2 and remove it
        if c2:
            p2 = c2.pop()
            data.append(p2)

print(data)


[5.899532887812484, 6.61864237271875, 5.206220910766344, 8.070172811242438, 7.2879378185693104, 4.870290673001325, 7.70641928914293, 5.71187504883994, 4.084481410417218]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Number of Components (K): The number of Gaussian components in the mixture.
Value: K = 2 (two mixture components).
Mixing Coefficients (π): The weights or probabilities associated with each component, indicating the likelihood of selecting a component.
Values: π1 and π2 (the actual values are not provided).
Component Means (μ): The mean vector for each component, representing the center of the Gaussian distribution.
Values: μ1 and μ2 (the actual values are not provided).
Component Covariance Matrices (Σ): The covariance matrices for each component, describing the spread and shape of the distribution.
Values: Σ1 and Σ2 (the actual values are not provided).

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
from sklearn.cluster import KMeans

# Fit a K-Means model with k=2
kmeans = KMeans(n_clusters=2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

# Separate data points into two clusters based on K-Means labels
c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]

# Estimate the proportion of points in each cluster (P(C_j))
prob_c = [len(c1) / (len(c1) + len(c2)), len(c2) / (len(c1) + len(c2))]

# Estimate the means of each cluster (mean_j)
mean = [sum(c1) / len(c1), sum(c2) / len(c2)]

# Estimate the variances of each cluster (variance_j)
var = [sum(map(lambda x: (x - mean[0])**2, c1)) / len(c1), sum(map(lambda x: (x - mean[1])**2, c2)) / len(c2)]

# Print the parameter estimates
print("P(C_1) = " + str(prob_c[0]) + ", P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ", mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ", var_2 = " + str(var[1]))


P(C_1) = 0.5555555555555556, P(C_2) = 0.4444444444444444
mean_1 = 5.154480186167462, mean_2 = 7.420793072918357
var_1 = 0.4188261446489722, var_2 = 0.29109316109487193


/Users/jamesxiao/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_c0_x = []  # P(C_0 | X_i)
prob_c1_x = []  # P(C_1 | X_i)

k = 2

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x_i = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x_i = (pdf_i[0] * prob_c[0]) / prob_x_i
    prob_c1_x_i = (pdf_i[1] * prob_c[1]) / prob_x_i

    prob_c0_x.append(prob_c0_x_i)
    prob_c1_x.append(prob_c1_x_i)

# Iterate over the computed probabilities and make a comment on cluster membership
for p, prob_c0, prob_c1 in zip(data, prob_c0_x, prob_c1_x):
    print("Point =", p)
    print("Probability of coming from C_1 =", prob_c0)
    print("Probability of coming from C_2 =", prob_c1)
    if prob_c0 > prob_c1:
        print("Belongs to Cluster 1")
    else:
        print("Belongs to Cluster 2")
    print()


Point = 5.899532887812484
Probability of coming from C_1 = 0.9999934283937861
Probability of coming from C_2 = 6.571606213876505e-06
Belongs to Cluster 1

Point = 6.61864237271875
Probability of coming from C_1 = 0.07911561208249299
Probability of coming from C_2 = 0.920884387917507
Belongs to Cluster 2

Point = 5.206220910766344
Probability of coming from C_1 = 0.9999999999996865
Probability of coming from C_2 = 3.135418043597016e-13
Belongs to Cluster 1

Point = 8.070172811242438
Probability of coming from C_1 = 3.1320034501418517e-10
Probability of coming from C_2 = 0.9999999996867996
Belongs to Cluster 2

Point = 7.2879378185693104
Probability of coming from C_1 = 2.236902858410256e-06
Probability of coming from C_2 = 0.9999977630971415
Belongs to Cluster 2

Point = 4.870290673001325
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 3.0962214976341344e-17
Belongs to Cluster 1

Point = 7.70641928914293
Probability of coming from C_1 = 1.2197257001127734e-08
Proba

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_c0_x) / len(prob_c0_x), sum(prob_c1_x) / len(prob_c1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [sum([x[1] * (x[0] - mean[0])**2 for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[1] * (x[0] - mean[1])**2 for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("Updated Estimates:")
print("P(C_1) = " + str(prob_c[0]) + ", P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ", mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ", var_2 = " + str(var[1]))


Updated Estimates:
P(C_1) = 0.5643456887031059, P(C_2) = 0.4356543112968942
mean_1 = 5.177286921480467, mean_2 = 7.436976338178086
var_1 = 244.1327776841843, var_2 = 678.9609740596803


g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
prob_c0_x = []  # P(C_0 | X_i)
prob_c1_x = []  # P(C_1 | X_i)

k = 2

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x_i = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x_i = (pdf_i[0] * prob_c[0]) / prob_x_i
    prob_c1_x_i = (pdf_i[1] * prob_c[1]) / prob_x_i

    prob_c0_x.append(prob_c0_x_i)
    prob_c1_x.append(prob_c1_x_i)

# Iterate over the computed probabilities and make a comment on cluster membership
for p, prob_c0, prob_c1 in zip(data, prob_c0_x, prob_c1_x):
    print("Point =", p)
    print("Updated Probability of coming from C_1 =", prob_c0)
    print("Updated Probability of coming from C_2 =", prob_c1)
    if prob_c0 > prob_c1:
        print("Likely belongs to Cluster 1")
    else:
        print("Likely belongs to Cluster 2")
    print()


Point = 5.899532887812484
Updated Probability of coming from C_1 = 0.7827334901542756
Updated Probability of coming from C_2 = 0.21726650984572432
Likely belongs to Cluster 1

Point = 6.61864237271875
Updated Probability of coming from C_1 = 0.7827309579549819
Updated Probability of coming from C_2 = 0.21726904204501807
Likely belongs to Cluster 1

Point = 5.206220910766344
Updated Probability of coming from C_1 = 0.782734715058031
Updated Probability of coming from C_2 = 0.21726528494196898
Likely belongs to Cluster 1

Point = 8.070172811242438
Updated Probability of coming from C_1 = 0.7827219325692611
Updated Probability of coming from C_2 = 0.2172780674307388
Likely belongs to Cluster 1

Point = 7.2879378185693104
Updated Probability of coming from C_1 = 0.7827274467891696
Updated Probability of coming from C_2 = 0.21727255321083044
Likely belongs to Cluster 1

Point = 4.870290673001325
Updated Probability of coming from C_1 = 0.7827348790588577
Updated Probability of coming from C

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [9]:
# Initialize an empty list for cluster assignments
cluster_assignments = []

# Iterate over the computed probabilities and make a hard assignment
for prob_c0, prob_c1 in zip(prob_c0_x, prob_c1_x):
    if prob_c0 > prob_c1:
        cluster_assignments.append(0)  # Assign to Cluster 0
    else:
        cluster_assignments.append(1)  # Assign to Cluster 1

# Print the cluster assignments for each point
print("Cluster Assignments:")
for i, assignment in enumerate(cluster_assignments):
    print(f"Point {data[i]} is assigned to Cluster {assignment}")


Cluster Assignments:
Point 5.899532887812484 is assigned to Cluster 0
Point 6.61864237271875 is assigned to Cluster 0
Point 5.206220910766344 is assigned to Cluster 0
Point 8.070172811242438 is assigned to Cluster 0
Point 7.2879378185693104 is assigned to Cluster 0
Point 4.870290673001325 is assigned to Cluster 0
Point 7.70641928914293 is assigned to Cluster 0
Point 5.71187504883994 is assigned to Cluster 0
Point 4.084481410417218 is assigned to Cluster 0


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      Y       |
| A  C |      Y       |
| A  D |      N       |
| A  E |      Y       |
| B  C |      N       |
| B  D |      Y       |
| B  E |      Y       |
| C  D |      Y       |
| C  E |      N       |
| D  E |      Y       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

The formula for calculating the number of unique pairs of points that can be created from N points is given by the binomial coefficient formula, often denoted as "n choose 2" or "C(n, 2)." It's expressed as:

\[C(n, 2) = \frac{n(n-1)}{2}\]

Where:
- \(C(n, 2)\) is the number of unique pairs of points.
- \(n\) is the total number of points.

This formula represents the number of ways to choose two distinct points from a set of \(n\) points, which gives you all possible unique pairs of points.

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

36

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

Let's determine the number of agreements and disagreements between clustering methods C and P for each cluster:

1. For Cluster 0 (P):
   - Number of pairs within Cluster 0 (P) that agree with C = \((3 \cdot 2) / 2 = 3\) pairs (as all points in C are in the same cluster).
   - Number of pairs within Cluster 0 (P) that disagree with C = \((3 \cdot 2) / 2 - 3 = 0\) pairs.

2. For Cluster 1 (P):
   - Number of pairs within Cluster 1 (P) that agree with C = \((2 \cdot 1) / 2 = 1\) pair.
   - Number of pairs within Cluster 1 (P) that disagree with C = \((2 \cdot 1) / 2 - 1 = 0\) pair.

3. For Cluster 2 (P):
   - Number of pairs within Cluster 2 (P) that agree with C = \((4 \cdot 3) / 2 = 6\) pairs.
   - Number of pairs within Cluster 2 (P) that disagree with C = \((4 \cdot 3) / 2 - 6 = 6\) pairs.

Now, let's calculate the total number of agreements and disagreements:

- Total agreements between C and P = \(3 + 1 + 6 = 10\) agreements.
- Total disagreements between C and P = \(0 + 0 + 6 = 6\) disagreements.

So, there are a total of 10 agreements and 6 disagreements between clustering methods C and P for all clusters.

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

To efficiently compute disagreement distances on large datasets, we can use a divide-and-conquer algorithm inspired by the process described above. This algorithm breaks down the dataset into smaller manageable chunks and computes the disagreements separately for each cluster. Here's an outline of the algorithm:

1. Divide the large dataset into smaller chunks or clusters based on a chosen criterion. The number of clusters or chunks should be reasonably small to ensure efficiency.

2. For each cluster:
   - Calculate the number of agreements and disagreements between clustering methods C and P within that cluster.
   - Keep track of these counts for each cluster.

3. Repeat steps 1 and 2 for all clusters or chunks.

4. Compute the total number of agreements and disagreements by summing the counts from all clusters.

5. Report the overall agreement and disagreement metrics.

By using this divide-and-conquer approach, we can efficiently compute disagreement distances on large datasets because we're processing smaller chunks of data at a time, reducing the computational complexity. Additionally, parallel processing or distributed computing can be employed to further speed up the computation of disagreements within each cluster. This approach allows for scalability and efficient handling of large datasets.